In [1]:
import pandas as pd

In [27]:
df = pd.DataFrame({'a':[100,200,100,200],'b':[100,200,300,400]})

In [28]:
df.head()

,a,b
0,100,100
1,200,200
2,100,300
3,200,400


In [29]:
df_calc = df.copy()

In [30]:
df_calc['percentage_sharp_drop'] = (((df.iloc[-2]['a'] - df.iloc[-1]['a']) / df.iloc[-2]['a'] ) * 100)

In [31]:
df_calc.head()

,a,b,percentage_sharp_drop
0,100,100,-100.0
1,200,200,-100.0
2,100,300,-100.0
3,200,400,-100.0


In [23]:
abs(df.iloc[-2]['a'] - df.iloc[-1]['a'] ) 

100

In [24]:
 df.iloc[-1]['a']

400

In [18]:
100/400

0.25

In [ ]:
1530

In [47]:
from pathlib import Path
from typing import List, Dict

def _load_tickers(path: str = "../../../resource/tickers.txt") -> List[str]:
    f = Path(path)
    if not f.exists():
        print("Nai desu :V")
        exit(1)
        return ["7203.T", "6758.T", "9984.T", "8306.T", "9432.T"]

    return [
        line.strip()
        for line in f.read_text(encoding="utf‑8").splitlines()
        if line.strip() and not line.lstrip().startswith("#")
    ]

In [48]:
list = _load_tickers()

In [49]:
batch_size = 15
increment = 0
while increment + batch_size < len(list):

    print(list[increment:increment + batch_size])

    increment += batch_size

print(list[increment:len(list)])

['1333.T', '1379.T', '1605.T', '1662.T', '1663.T', '1719.T', '1721.T', '1801.T', '1802.T', '1803.T', '1808.T', '1812.T', '1820.T', '1860.T', '1893.T']
['1911.T', '1925.T', '1928.T', '1942.T', '1944.T', '1951.T', '1959.T', '1963.T', '2002.T', '2201.T', '2206.T', '2212.T', '2222.T', '2229.T', '2264.T']
['2267.T', '2269.T', '2282.T', '2501.T', '2502.T', '2503.T', '2531.T', '2579.T', '2587.T', '2593.T', '2607.T', '2801.T', '2802.T', '2809.T', '2810.T']
['2811.T', '2871.T', '2875.T', '2897.T', '2914.T', '3101.T', '3401.T', '3402.T', '3405.T', '3407.T', '3436.T', '3861.T', '3863.T', '3865.T', '3941.T']
['4004.T', '4005.T', '4021.T', '4042.T', '4043.T', '4061.T', '4063.T', '4063.T', '4088.T', '4091.T', '4118.T', '4151.T', '4182.T', '4183.T', '4186.T']
['4188.T', '4202.T', '4203.T', '4204.T', '4205.T', '4208.T', '4272.T', '4452.T', '4502.T', '4502.T', '4503.T', '4506.T', '4507.T', '4516.T', '4519.T']
['4523.T', '4527.T', '4528.T', '4530.T', '4536.T', '4540.T', '4552.T', '4565.T', '4568.T', '45

In [55]:
!pip install yfinance

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [150]:
# data_collector.py – v3 (file‑based persistence)
from __future__ import annotations
from typing import List, Dict, Tuple, Set
from datetime import datetime
from zoneinfo import ZoneInfo
from pathlib import Path

import csv

import yfinance as yf
import pandas as pd
import ta


# ──────────────────────────────────────────────────────────────────────────────
# Configuration
# ──────────────────────────────────────────────────────────────────────────────

TZ = ZoneInfo("Asia/Tokyo")
TODAY: str = datetime.now(TZ).date().isoformat()  # e.g., "2025-05-11"
# ALERTS_FILE: Path = Path(__file__).with_name("data/alerts_log.csv")
ALERTS_FILE: Path = Path("/home/mhr/Documents/programming/trading-bot/tse-trading-bot/devs/pocs/pandas_test/data/alerts_log.csv")


# ──────────────────────────────────────────────────────────────────────────────
# Lightweight file‑based persistence (CSV)
# ──────────────────────────────────────────────────────────────────────────────

# Row schema: alert_date,ticker,alert_type
_alert_cache: Set[Tuple[str, str, str, int]] | None = None


def _load_alert_cache() -> None:
    global _alert_cache
    if _alert_cache is not None:
        return  # already loaded

    _alert_cache = set()
    if ALERTS_FILE.exists():
        with ALERTS_FILE.open("r", newline="") as f:
            reader = csv.reader(f)
            for row in reader:
                if len(row) == 4:
                    _alert_cache.add(tuple(row))


def _already_alerted(ticker: str, alert_type: str, price: int) -> bool:
    _load_alert_cache()
    assert _alert_cache is not None
    return (TODAY, ticker, alert_type) in _alert_cache


def _mark_alert(ticker: str, alert_type: str, price: int) -> None:
    _load_alert_cache()
    assert _alert_cache is not None
    key = (TODAY, ticker, alert_type, price)
    if key in _alert_cache:
        return  # already recorded

    # Append to file first (so even if script crashes later we don’t lose the entry)
    with ALERTS_FILE.open("a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(key)

    _alert_cache.add(key)

In [151]:
results = []
results.append(
                {
                    "Ticker": "12312.T",
                    "BUY_SIGNAL":"BUY",
                    "Price": 123.12,
                    "BuySignal": "Confluence",
                    "RSI": 45,
                    "MACD Signal": "Buy", 
                    "Support": 120.11,
                    "Resistance": 170.22,
                    "Name": "name",
                }
            )

In [152]:
results[0]['Ticker']

'12312.T'

In [155]:
_already_alerted(ticker=results[0]['Ticker'],
                 alert_type="buy",
                 price=int(results[0]['Price']))

False

In [154]:
_mark_alert(ticker=results[0]['Ticker'],
                 alert_type="buy",
                 price=int(results[0]['Price']))